In [7]:
#Importing all the libraries used in the calculation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon
from datetime import datetime,timedelta
from math import sin, cos, sqrt, atan2, radians, ceil
import folium
from geopy.geocoders import Nominatim
from folium import plugins
from folium.plugins import MarkerCluster
import ipywidgets as widgets
from IPython.display import clear_output

#Configuring plots
plt.style.use('seaborn')

plt.rcParams.update({'font.size': 15})
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 18

week_days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

In [2]:
#Loading shape files and tables to save up computing time
lookout_df = gpd.read_file('radius_files/crime_data_kommune.shp') #All break-ins within the last year with coordinates and kommune, taken from lookout's database api
cit = gpd.read_file('radius_files/KOMMUNE.shp') #All Kommune shapefiles (provided in UTM32 Coordinate Reference System)
denmark_grid = pd.read_excel('radius_files/denmark_score_grid_2019-2020.xlsx') #Grid in Denmark of 1km² with the amount of burglaries per square in the grid
home_burg = pd.read_excel('radius_files/Quarterly_Burglaries.xlsx').set_index('Kommune') #Quarterly amount of residential break-ins in Denmark from Denmarks Statistik
lookout_home_burg = pd.read_excel('radius_files/local_home_burg_2019Q3-2020Q2.xlsx') #4 quarters of lookout's data (taken along with the last 4 quarters available from DK Statistik)
weekly_ranking = pd.read_excel('radius_files/weekly_ranking.xlsx').set_index('Week') #Ranking of weeks with most burglaries in 2020

total_last_year = home_burg[["2019Q3","2019Q4","2020Q1","2020Q2"]].T.sum().reindex(home_burg.index) #Total number of burglaries in last year of available data
lookout_last_year = lookout_home_burg.groupby('Kommuner').count()['Coord'] #Total number of burglaries in lookout's database during the same period
per1000_quarterly= pd.read_excel('radius_files/Home_BurglariesPer1000.xlsx').set_index('Kommune') #Burglaries per 1000 inhabitants (DK Statistik)


#Only get kommunes that we have 1/3 or more of the data (counting from Q3-2019 to Q2-2020)
ratio_to_total = lookout_last_year/total_last_year
usable_data = ratio_to_total[ratio_to_total>1/3]

WSG_breakin = lookout_df['geometry'].copy() #Creating a copy for lighter manipulation of table and setting up the Coordinate Reference System (crs)
cit_latlon = cit.to_crs('EPSG:4326')
WSG_breakin.crs = 'EPSG:4326'

geolocator = Nominatim(user_agent="LookoutAnalytics") #Starting Folium map (Limited license)


#Adding weekday column to dataframe
lookout_df['weekday'] = [week_days[day.weekday()] for day in pd.to_datetime(lookout_df['committed_'])] 


In [3]:
#Setting up functions for map printing
def get_radius_coord(lon,lat, radius):
    # approximate radius of earth in km
    R = 6373.0
    pi = 3.14159

    max_lat = lat + (radius/ R) * (180 / pi);
    max_lon = lon + (radius/ R) * (180 / pi) / cos(lat * pi/180);
    min_lat = lat - (radius/ R) * (180 / pi);
    min_lon = lon - (radius/ R) * (180 / pi) / cos(lat * pi/180);
    return min_lon, min_lat, max_lon, max_lat

def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

#setting up quantiles table for radius risk calculation
quant_table = denmark_grid.groupby(by='Kommune').aggregate(['mean',percentile(10),percentile(20),percentile(30),percentile(40),percentile(50),percentile(60),
                                                                percentile(70),percentile(80),percentile(90)])

def calculate_risks(location, date):
        #Calculating Radius Risk based on the global quantiles table
    min_lon, min_lat, max_lon, max_lat = get_radius_coord(location.longitude,location.latitude, 5)
    area_5km = Polygon([[min_lon, min_lat],[min_lon, max_lat],[max_lon, max_lat],[max_lon, min_lat]])

    quantiles_table = quant_table.reindex(usable_data.index)

    address_breakins = sum(WSG_breakin.within(area_5km))
    if "Kommune" in location.address:
        kommune = location.address.split('Kommune')[0].replace(' ','').split(',')[-1]
    else:
        kommune = location.address.split(',')[0]

    
    #Handling possible differences in the kommune's name (Københavns Kommune != København Kommune)
    if kommune in quantiles_table.index or kommune+"s" in quantiles_table.index:
        try:
            quantile_row = quantiles_table["Burglaries_5km"].loc[kommune].tolist()[1:]

        except KeyError:
            quantile_row = quantiles_table["Burglaries_5km"].loc[kommune+"s"].tolist()[1:]   
            kommune = kommune[:-1]

        idx, value = min(enumerate(quantile_row), key=lambda x: abs(x[1]-address_breakins))
        if address_breakins<=value:
            rad_risk = idx+1
        else:
            rad_risk = idx+2    
    else:
        rad_risk = False

    #Weekday Risk based on today's date and historical data (Lookout's database)
    daily_risk = lookout_df['weekday'].value_counts()/lookout_df['weekday'].value_counts().mean() * 6
    today_name = week_days[date.weekday()]
    weekday_risk = round(daily_risk[today_name])

    #Recent Burglary Risk based on burglaries that happened within 1km radius of location in the last 21 days (Lookout's Database)
    min_lon, min_lat, max_lon, max_lat = get_radius_coord(location.longitude,location.latitude, 1)
    area_1km = Polygon([[min_lon, min_lat],[min_lon, max_lat],[max_lon, max_lat],[max_lon, min_lat]])

    break_ins_in_area = lookout_df.reset_index()[WSG_breakin.within(area_1km)].copy()
    dates_break_ins_area  = pd.to_datetime(break_ins_in_area['committed_'])
    recent_dates = dates_break_ins_area[dates_break_ins_area<=date].tolist()
    
    if len(recent_dates)>0:
        recent_dates.sort()
        interval = date - recent_dates[-1]
        recent_risk = round(max([(22 - interval.days)/2.1,1]))
    else:
        recent_risk = 0.99

    #Kommune Risk based on the burglaries per 1000 inhabitants in the last year of available data for each kommune
    last_year = per1000_quarterly.iloc[:,-4:]
    per1000_rank = last_year.mean(axis = 1).sort_values()
    try:
        kom_risk = (per1000_rank[kommune] - per1000_rank[0])/((per1000_rank[-1] - per1000_rank[0]))*9 + 1
    except:
        kommune = kommune[:-1]
        kom_risk = (per1000_rank[kommune] - per1000_rank[0])/((per1000_rank[-1] - per1000_rank[0]))*9 + 1

    #Quarter Risk calculated with the current quarter and average of the current quarter over the last 5 years (DK Statistik data)
    burg_last_5years = home_burg.copy().iloc[:,-20:]
    Q1,Q2,Q3,Q4 = 0,0,0,0

    for i,item in enumerate(burg_last_5years):
        if "Q1" in item:
            Q1 += burg_last_5years.loc[kommune].iloc[i]
        elif "Q2" in item:
            Q2 += burg_last_5years.loc[kommune].iloc[i]
        elif "Q3" in item:
            Q3 += burg_last_5years.loc[kommune].iloc[i]
        elif "Q4" in item:
            Q4 += burg_last_5years.loc[kommune].iloc[i]
            
    current_quarter = "Q"+str(int(ceil(date.month/3)))
    quarterly_sum = {"Q1":Q1, "Q2":Q2, "Q3":Q3, "Q4":Q4}
    quarter_risk = quarterly_sum[current_quarter]/(sum(quarterly_sum.values())/4)*4.5 + 1

    #Weekly Risk calculated using lookout's database from 2020
    current_week = date.isocalendar()[1]
        
    week_variation = (weekly_ranking['Weekly Break-ins'].max() - weekly_ranking['Weekly Break-ins'].min())
    dif_from_minimum_week = (weekly_ranking.loc[current_week]['Weekly Break-ins'] - weekly_ranking['Weekly Break-ins'].min())
    week_risk = 9*dif_from_minimum_week/week_variation+1

    #Overall Risk taken as a weighted average of all risks calculated above (when available)
    if rad_risk == False:
        all_risks = [0, weekday_risk, recent_risk, kom_risk, quarter_risk, week_risk]
        risk_weights = [0, 0.5, 3, 2, 1, 0.5]
    else:
        all_risks = [rad_risk, weekday_risk, recent_risk, kom_risk, quarter_risk, week_risk]
        risk_weights = [1, 0.5, 3, 2, 1, 0.5]
    overall_risk = round((np.array(all_risks)*np.array(risk_weights)).sum()/np.array(risk_weights).sum(),2)
    
    risk_data = all_risks + [overall_risk] + [recent_dates] + [today_name] + [kommune] + [current_week] 
    return risk_data

def risk_timeline(location, n_days):
    global dates,overall_risks
    dates = [datetime.today() - timedelta(days=i) for i in range(n_days,0,-1)]
    overall_risks = [calculate_risks(location, current_date)[6] for current_date in dates]
    fig = plt.figure(figsize=(16,6))

    plt.plot(np.array(dates), np.array(overall_risks))
    plt.xlabel('Date', fontsize = 18)
    plt.ylabel('Risk', fontsize = 18)
    plt.title('Risk Timeline', fontsize = 20)
    plt.show()

In [4]:
def print_map(address, radius):

    #Handling Exceptions with radius and address
    try:
        radius = int(radius)/1000
    except:
        return print('It has to be a number :)')

    location = geolocator.geocode(address,country_codes = 'dk')
    
    try:
        min_lon, min_lat, max_lon, max_lat = get_radius_coord(location.longitude,location.latitude, radius)
    except:
        return print('Address not found')
    
    #Defining area to search for crimes
    area = Polygon([[min_lon, min_lat],[min_lon, max_lat],[max_lon, max_lat],[max_lon, min_lat]])
    
    #Defining area that will display crimes (Done to save time)
    big_min_lon, big_min_lat, big_max_lon, big_max_lat = get_radius_coord(location.longitude,location.latitude, radius*2)
    big_area = Polygon([[big_min_lon, big_min_lat],[big_min_lon, big_max_lat],[big_max_lon, big_max_lat],[big_max_lon, big_min_lat]])

    #Initializing Map Plot and Setting up markers for each burglary
    city_map = folium.Map(location = [location.latitude, location.longitude],
                         zoom_start = 13,
                         tiles = 'openstreetmap')

    for p in WSG_breakin[WSG_breakin.within(big_area)].geometry:
        lat = p.y
        lon = p.x
        marker = folium.Marker([lat,lon],icon=folium.Icon(color="red")).add_to(city_map)

    #Printing Home Marker with Address in pop-up
    popup_text = folium.Popup(location.address, parse_html = True,max_width=300,min_width=300)
    home_marker = folium.Marker([location.latitude, location.longitude], popup=popup_text, icon=folium.Icon(color="blue", icon="home")).add_to(city_map)

    b1,b2 = area.exterior.bounds[0:2], area.exterior.bounds[2:]
    lon1, lat1 = b1[0], b1[1]
    lon2, lat2 = b2[0], b2[1]
    
    #Plotting Orange square along the selected "radius"
    folium.Rectangle(bounds=[(lat1,lon1),(lat2,lon2)], color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.2).add_to(city_map)

    #Counting the number of break-ins within selected area
    n_breakins = sum(WSG_breakin.within(area))
    folium_title = f'{n_breakins} break-ins found in the selected area'
    city_map.get_root().add_child(folium.Element(folium_title))
    
    #Calculating and printing each risk for today's date (in datetime format)
    todays_date = datetime.today()
    [rad_risk, weekday_risk, recent_risk, kom_risk, quarter_risk, week_risk, overall_risk, recent_dates, today_name, kommune, current_week] = calculate_risks(location,todays_date)
    
    print("\n"+folium_title)
   
    if rad_risk > 0:
        print(f"\nThe radius-based risk for {location.address} is: {rad_risk}")
    else:
        print(f'\nLookout does not have 33% or more of break-in data for {kommune} Kommune')
  
    print(f'\nThe risk for {today_name} is {weekday_risk}')
    
    if recent_risk > 0.99:
        print(f'\nThe last burglary within 1km radius was on {recent_dates[-1].strftime("%d of %b, %Y")} (risk: {recent_risk})')
    else:
        recent_risk = 0.99
        print(f'\nNo burglaries were found within a 1km radius of the area (risk:  {round(recent_risk)})')

    print(f'\nThe Kommune-based risk for {kommune} is {round(kom_risk,2)}')
    
    print(f'\nThe risk in Denmark for week {current_week} is {round(week_risk,2)} if compared to other weeks')
    
    print(f'\nThe overall risk for your location is  {overall_risk}')
    
    #plotting the map
    display(city_map)
    
    risk_timeline(location, 180)

In [5]:
button = widgets.Button(description='Calculate Risk')
address = widgets.Text(value = '', placeholder="Type some address in Denmark", description = "Address: ")
radius = widgets.Text(value = '', placeholder="Type some radius [m]", description = "Radius: ")

out = widgets.Output()
def on_button_clicked(_):
      # "linking function with output"
    with out:
          # what happens when we press the button
        clear_output()
        print_map(address.value, radius.value)
# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([address,radius, button,out])